In [3]:
import pynq
import numpy as np
import sys
import time


ol = pynq.Overlay('./design_1.bit')
mmio = ol.LEA_En_192_0.mmio


def ndarray_from_mmio(name, size, dtype):
    reginfo = ol.ip_dict['LEA_En_192_0']['registers'][name]
    addr_start = reginfo['address_offset'] // 4
    addr_end = addr_start + reginfo['size'] // 4
    mmio_array = mmio.array[addr_start:addr_end]
    mmio_array.dtype = np.uint32
    return mmio_array.reshape(size)

def ndarray_from_mmio_2(name, size, dtype):
    reginfo = ol.ip_dict['LEA_En_192_0']['registers'][name]
    addr_start = reginfo['address_offset'] // 4
    addr_end = addr_start + reginfo['size'] // 5
    mmio_array = mmio.array[addr_start:addr_end]
    mmio_array.dtype = np.uint32
    return mmio_array.reshape(size)


mmio_ciphertext = ndarray_from_mmio(
    'Memory_Cipher', size=4, dtype=np.uint32)
mmio_plaintext = ndarray_from_mmio('Memory_Plain', size=4, dtype=np.uint32)
mmio_key = ndarray_from_mmio_2('Memory_Key_K', size=6, dtype=np.uint32)


def LEA192Enc_fpga(ciphertext: np.ndarray, plaintext: np.ndarray, key: np.ndarray):
    mmio_plaintext[:] = plaintext
    mmio_key[:] = key
    mmio.write(0, 1)
    # 0x02 means DONE.
    while not (mmio.read(0) & 0x02):
        pass


# Encrypt a test vector using FPGA
# ISO/IEC 29192-2, p.49
key = np.array([0x3c2d1e0f, 0x78695a4b, 0xb4a59687, 0xf0e1d2c3, 0xc3d2e1f0, 0x8796a5b4])
plaintext = np.array([0x23222120, 0x27262524, 0x2b2a2928, 0x2f2e2d2c])
ciphertext = np.array([0x325eb96f, 0x871bad5a,0x35f5dc8c,0xf2c67476])

LEA192Enc_fpga(mmio_ciphertext, plaintext, key)
ciphertext_fpga = mmio_ciphertext.copy()
np.set_printoptions(formatter={'int': '{:x}'.format})
print("Cipher = ", end="")
print(ciphertext_fpga)

Cipher = [325eb96f 871bad5a 35f5dc8c f2c67476]


In [4]:
# Check the result
for i in range(0, 4):
    if ciphertext[i] != ciphertext_fpga[i]:
        print("NG")
        for j in range(0, 4):
            print(str(j) + '  ' + format(ciphertext[j], '#08x') +
                  ' ' + format(ciphertext_fpga[j], '#08x'), end='')
            if j == i:
                print(' xxxxxxx', flush=True)
            else:
                print('', flush=True)
        sys.exit(1)

print("OK", flush=True)
# Measure execution time.
NUM_OF_LOOPS = 1 << 17
i = 0
start_time = time.perf_counter_ns()
start_time_cpu = time.process_time_ns()
while i < NUM_OF_LOOPS:
    LEA192Enc_fpga(mmio_ciphertext, plaintext, key)
    i += 1
end_time_cpu = time.process_time_ns()
end_time = time.perf_counter_ns()
elapsed_time_ns = end_time - start_time
elapsed_time_cpu_ns = end_time_cpu - start_time_cpu
print('Num ' + str(NUM_OF_LOOPS))
print('Total ' + str(elapsed_time_ns) + ' ns')
print('CPU   ' + str(elapsed_time_cpu_ns) + ' ns')
print('Per 1 time ')
print('Total ' + '{} ns'.format((end_time -
      start_time) / NUM_OF_LOOPS))
print('CPU   ' + '{} ns'.format((end_time_cpu -
      start_time_cpu) / NUM_OF_LOOPS))

OK
Num 131072
Total 8302298635 ns
CPU   8290768158 ns
Per 1 time 
Total 63341.51180267334 ns
CPU   63253.541244506836 ns
